In [52]:
#..........................案例1:
#.......................... 集成学习：群体智慧
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
X,y=make_moons(n_samples=100)
dta=pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1)
trian_s,test_s=train_test_split(dta,test_size=0.2,random_state=20) #random_state相当于seed
# train_test_split可以同时split x y; X_train, X_val, y_train, y_val = train_test_split(X, y)
trian_X,trian_y=trian_s.iloc[:,[0,1]],trian_s.iloc[:,2]
test_X,test_y=test_s.iloc[:,[0,1]],test_s.iloc[:,2]
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf),('svc', svm_clf)],voting='hard')
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(trian_X, trian_y)
    y_pred = clf.predict(test_X)
    print(clf.__class__.__name__, accuracy_score(test_y, y_pred))
# 每次结果不太一样，Vot一般较好，SVM输出的不是概率，无法直接比较。
# voting="soft" 来保证分类器可以预测类别概率，经常比硬投票表现的更好
# 然而这不是 SVC类的分类器默认的选项，所以你需要把它的 probability hyperparameter 设置为 True （这会使 SVC 使用交叉验证去预测类别概率，其降低了训练速度，但会添加 predict_proba() 方法）

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 1.0
VotingClassifier 0.95


In [59]:
# 集成算法
# Bagging 和 Pasting
# Bagging 和 Pasting 都允许在多个分类器间对训练集进行多次采样，Bagging 允许对同一种分类器上对训练集进行进行多次采样
# 有放回采样被称为装袋（Bagging，是 bootstrap aggregating 的缩写）。无放回采样称为粘贴（pasting）。
# 500 个决策树分类器的集成
# 有放回采样 80 个训练实例下进行训练
from sklearn.ensemble import BaggingClassifier
# 如果拥有predict_proba()，BaggingClassifier会自动软投票
# BaggingRegressor回归
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,max_samples=80, bootstrap=True, n_jobs=-1,oob_score=True)
# oob_score=True使得样本自动交叉验证
# -1代表着 sklearn 会使用所有空闲核，CPU核
# 如果你想尝试Pasting，就设置 bootstrap=False
bag_clf.fit(trian_X, trian_y)
y_pred=bag_clf.predict(test_X)
# Bagging能减少方差，相对于Pasting，Bagging一般更好
print('自动交叉验证的准确率accuracy（结果一般与最后交叉验证相似）：',bag_clf.oob_score_)
# bag_clf.oob_decision_function_  # 训练实例的概率
# BaggingClassifier支持特征采样，由max_features 和 bootstrap_features 控制

自动交叉验证的准确率accuracy（结果一般与最后交叉验证相似）： 0.9375


In [60]:
# 随机森林
# 决策树的集成：通过 bagging 方法（有时是pasting 方法）
# 与上面随机森林不同，更方便的用决策树优化够的 RandomForestClassifier
# RandomForestClassifier集成了大部分DecisionTreeClassifier和BaggingClassifier的超参数
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(trian_X, trian_y)
y_pred_rf = rnd_clf.predict(test_X)
# 大致相当于：
# bag_clf = BaggingClassifier(DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)
# 与在节点分裂时需要找到最好分裂特征相反，它在一个随机的特征集中找最好的特征。它导致了树的差异性，并且再一次用高偏差换低方差，总的来说是一个更好的模型。
# 当你在随机森林上生长树时，在每个结点分裂时只考虑随机特征集上的特征。相比于找到更好的特征我们可以通过使用对特征使用随机阈值使树更加随机（像规则决策树一样）。
print('特征重要性（顺序与变量一致）：',rnd_clf.feature_importances_)

特征重要性（顺序与变量一致）： [0.42963793 0.57036207]


In [71]:
# Adaboost提升：几个弱学习者组合成强学习者的集成方法
# 给每个“实例”一定的权重（m个实例为1/m），分错的点加权重，不断地重复，最后进行集成，集成与bagging类似
from sklearn.ensemble import AdaBoostClassifier
# AdaBoostRegressor回归
# 多分类版本 SAMME（这就代表了 分段加建模使用多类指数损失函数）。如果只有两类别，那么 SAMME 是与 Adaboost 相同的。
# 如果预测带有概率，效果会更好
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200,
algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(trian_X, trian_y)
print('预测结果：',ada_clf.predict_proba(test_X.iloc[0:2,:]))

预测结果： [[0.84610901 0.15389099]
 [0.00539119 0.99460881]]


In [72]:
# 提升算法之梯度提升
# 梯度提升也是通过向集成中逐步增加分类器运行的，每一个分类器都修正之前的分类结果。
# 不断使用新的分类器去拟合前面分类器预测的残差
# 原理如下：
from sklearn.tree import DecisionTreeRegressor
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(trian_X, trian_y)
y2 = trian_y - tree_reg1.predict(trian_X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(trian_X, y2)
y_pred = sum(tree.predict(test_X) for tree in (tree_reg1, tree_reg2))
# 更方便的做法：
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, learning_rate=1.0)
gbrt.fit(trian_X, trian_y)
# 学习率低可能过拟合，可以使用早停技术（第四章讨论）。最简单使用这个技术的方法就是使用 staged_predict()
# 每个阶段返回一个迭代器(先前的合计)
errors = [mean_squared_error(test_y, y_pred) for y_pred in gbrt.staged_predict(test_X)]
bst_n_estimators = np.argmin(errors)
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(trian_X, trian_y)
# 还可以通过设置warm_start=True来早停，
# 使得当 fit() 方法被调用时 sklearn 保留现有树，并允许增量(增加迭代次数)训练。
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)
min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(trian_X, trian_y)
    y_pred = gbrt.predict(test_X)
    val_error = mean_squared_error(test_y, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
    if error_going_up == 5:
        break # early stopping
# GradientBoostingRegressor支持指定每棵树训练实例比例subsample=0.25，
# 会随机选择25%的实例，即随机梯度提升，也是个高偏差换低方差的作用
# Stacking
# 主要原理：将预测的结果再进行机器学习






GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=22,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)